In [ ]:
#Autoreload so you don't have to restart the kernel every time you make a change to the source code
%load_ext autoreload
%autoreload 2

### 1: Generate a pandas dataframe containing convex hull data ###

In [ ]:
import CHQuant_test

In [ ]:
from CHQuant_test.data import df_from_path
# import pandas as pd

#Path to XYZ trajectory
#The trajectory should be aligned (Bienfait's metadynamics scripts do this automatically)
#You can also put a path to a directory containing XYZs for the same molecule (atom ordering needs to be the same)
input_path = "example_data/paracetamol_xtb300K_rotated.xyz"

df = df_from_path(input_path)
df
# df.to_csv("example_from_xyz.csv") #Save as CSV

In [ ]:
from CHQuant.data import df_from_path
import pandas as pd

#Path to XYZ trajectory
#The trajectory should be aligned (Bienfait's metadynamics scripts do this automatically)
#You can also put a path to a directory containing XYZs for the same molecule (atom ordering needs to be the same)
input_path = "example_data/paracetamol_xtb300K_rotated.xyz"

df = df_from_path(input_path)
df
# df.to_csv("example_from_xyz.csv") #Save as CSV

### 2: Generate and save convex hull vertices and atomic point clouds ###

In [ ]:
#Generate and save vertices
from CHQuant.vertices import generate_atomic_convex_hulls, save_vertices_points

#Path to XYZ trajectory
input_path = "example_data/paracetamol_xtb300K_rotated.xyz"

#Output directory
output_dir = "example_data/example_vertices"

#List of atoms (or edit to whatever you want the vertices CSVs to be called)
atomcount = int(open(input_path).readline())
atomlist = [a.split()[0]+str(i-1) for (i, a) in enumerate(open(input_path)) if i < atomcount+2 and i > 1]

#Generate atomic convex hulls (list of scipy convexhull objects)
atomic_convex_hulls = generate_atomic_convex_hulls(input_path)

#Save atomic convex hull vertices
for hull,filename in zip(atomic_convex_hulls,atomlist):
    save_vertices_points(hull, filename, output_dir)

### 3: Generating a pandas dataframe containing convex hull data from vertices ###

In [ ]:
from CHQuant.data import df_from_points
from CHQuant.vertices import load_directory_convex

#Path to a directory containing CSVs of atomic convex hull vertices
# input_path = "example_data/paracetamol_xtb300K_vertices/"
input_path = "example_data/example_vertices/"

system_name = "Paracetamol xTB 300K"

atom_names, point_clouds, hull_vertices = load_directory_convex(input_path)
df = df_from_points(point_clouds, system_name,atom_names)
df
# df.to_csv("example_from_points.csv")

### 4: Check atomic CHs for overlaps ###

In [ ]:
#Checking functional group list
#Edit the file to add new molecules & functional groups
from CHQuant.functional_group_list import molecule_atomcounts, molecule_functional_groups

print(molecule_functional_groups["paracetamol"])
# print(molecule_atomcounts)

In [ ]:
from CHQuant.functional_group_list import molecule_atomcounts, molecule_functional_groups
from CHQuant.functional_groups import functional_group_overlap_check

#Functional group names, atom indices are currently stored in functional_group_list.py

#Path to directory containing vertices CSVs
input_path = "example_data/example_vertices/"

#Molecule name
molecule_name = "paracetamol"

#Index of functional group to check
# functional_group = "Methyl"
functional_group_index = 0

functional_group_overlap_check(input_path, molecule_functional_groups[molecule_name][0][functional_group_index])

#change functional group list to dicts instead of separate lists...?? Or use zip to go through them?
#should FGs be numbered? i.e. methyl1, methyl2...

In [ ]:
molecule_functional_groups[molecule_name][0]

### 5: Print functional group CH volumes ###

In [ ]:
#Note that there are two definitions of FG volume:
#Sum of atomic CH volumes -> used when the atomic CHs don't overlap
#Volume of CH containing all atoms' positions pooled together -> used when the atomic CHs do overlap
#If using the second definition, the total volume will change as well

In [ ]:
from CHQuant.functional_groups import functional_group_nonoverlapping_volume, functional_group_overlapping_volume
from CHQuant.functional_group_list import molecule_functional_groups
import pandas as pd

#Name of molecule in functional_group_list.py
molecule_name = "paracetamol"

#Path to XYZ trajectory
input_path = "example_data/paracetamol_xtb300K_rotated.xyz"


fg_atom_indices, fg_names = molecule_functional_groups[molecule_name]
df_rows = []

for indices, name in zip(fg_atom_indices, fg_names):
    sum_volume = functional_group_nonoverlapping_volume(input_path, indices)
    pool_volume = functional_group_overlapping_volume(input_path, indices)
    
    new_row = {"Functional group name":name,
               "Atom indices":",".join([str(i) for i in indices]),
              "Sum volume": sum_volume,
              "Pool volume": pool_volume}
    
    df_rows.append(new_row)

#     print(indices)
#     print(name)
#     print(sum_volume)
#     print(pool_volume)
df = pd.DataFrame(df_rows)
df
# df.to_csv("example_functional_groups.csv")

### 6: Generate figure with mist plot and atomic CHs ###

In [ ]:
#Not included in module, hull_plot_functions.py is separate 
#also uses ICHOR; alphashape if you want to do concave hulls, scipy, sklearn if you want dimensionality reduction

from CHQuant.vertices import load_directory_convex
from Scripts.hull_plot_functions import plot_loaded_convex
from Scripts.atom_colouring import add_molecule_spheres

import plotly.graph_objects as go

#Path to directory containing vertices CSVs
input_path = "example_data/example_vertices/"

#Path to XYZ trajectory
geom_path = "example_data/paracetamol_xtb300K_rotated.xyz"

atom_names, point_clouds, hull_vertices = load_directory_convex(input_path)

fig = go.Figure()
fig = add_molecule_spheres(fig, geom_path, 0, 0.2, 20)
fig = plot_loaded_convex(fig,atom_names, point_clouds, hull_vertices)
fig
# fig.update(layout_showlegend=False)
# fig.update_layout(coloraxis_showscale=False)

fig.update_layout(
        scene = dict(
            xaxis = dict( range=[-8,8],visible=False),
            yaxis = dict( range=[-8,8],visible=False),
            zaxis = dict( range=[-8,8],visible=False),),)

fig

# img_bytes = fig.to_image(format="png",scale=4,width=1600, height=1000) #scale adjusts resolution
# with open(f"paracetamol_test.png", "wb") as file:
#     file.write(img_bytes)

### 7: Generate figure with atom colouring based on atomic CH volume ###

In [ ]:
#Not included in module, atom_colouring.py is separate
#Requires plotly (for generating plots/figures) and ICHOR (for connectivity)
#See atom_colouring_example notebook for more examples/functions
#Pymol might be better
#There are also some lines you can comment/uncomment in the script itself

from Scripts.atom_colouring import ms, matplotlib_to_plotly, add_molecule_spheres_volumescale
import pandas as pd
import numpy as np
import plotly.graph_objects as go

#Path to CH data CSV
input_path = "example_data/example.csv"

#Path to XYZ structure to use for the figure
mol_path = "example_data/paracetamol_frame0.xyz"

df = pd.read_csv(input_path, index_col=0)

#You'll need to edit this section to match your CSV structure
#example_values should be a 1D numpy array
example_values = df["paracetamol_xtb300K_rotated"][6:26].values

#This figure will be interactive in a notebook, or you can uncomment the later lines to output an image

fig = go.Figure()
fig = add_molecule_spheres_volumescale(fig, mol_path, example_values, "turbo")
fig

#If the atoms don't look spherical/the figure is cut off, adjust the axis ranges
#Uneven ranges will make the spheres look distorted
fig.update_layout(
        scene = dict(
            xaxis = dict( range=[-8,8],visible=False),
            yaxis = dict( range=[-8,8],visible=False),
            zaxis = dict( range=[-8,8],visible=False),),)

fig
#Save image
# fig.write_image("fig.png") 

# fig.write_image(f"fig.png",format="png",engine="kaleido")

# img_bytes = fig.to_image(format="png",scale=4,width=1600, height=1000) #scale adjusts resolution
# with open(f"fig_colorbar.png", "wb") as file:
#     file.write(img_bytes)